In [1]:
#dataset
#https://s3.amazonaws.com/fast-ai-nlp/amazon_review_full_csv.tgz
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
df = pd.read_csv('/home/marivishnu/PC/NLTK/Product Reviews/amazon_review_full_csv/test.csv')

positive = []
negative = []
for i in range(len(df)):
    if df.loc[i, "rating"] >= 4:
        positive.append(word_tokenize(str(df.loc[i, 'review2'])))
    if df.loc[i, "rating"] <= 2:
        negative.append(word_tokenize(str(df.loc[i, 'review2'])))
print(len(positive)) #260000
print(len(negative)) #260000

260000
260000


In [2]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string

def clean(words):
    lemmatizer = WordNetLemmatizer()
    stopwords_set = stopwords.words('english')
    
    clean_words = []
    for word in words:
        word = word.lower()
        if word not in stopwords_set and word not in string.punctuation:
            clean_words.append(lemmatizer.lemmatize(word))
    dict_words = dict([word, True] for word in clean_words)
    return dict_words

In [3]:
pos_set = []
for review in positive:
    pos_set.append((clean(review), 'pos'))
neg_set = []
for review in negative:
    neg_set.append((clean(review), 'neg'))

print(len(pos_set))
print(len(neg_set))
print(pos_set[0])
print(neg_set[0])

260000
260000
({'fast': True, 'read': True, 'filled': True, 'unexpected': True, 'humour': True, 'profound': True, 'insight': True, 'art': True, 'politics': True, 'policy': True, 'brief': True, 'sly': True, 'wry': True, 'wise': True}, 'pos')
({'model': True, 'may': True, 'ok': True, 'sedentary': True, 'type': True, "'m": True, 'active': True, 'get': True, 'around': True, 'alot': True, 'job': True, 'consistently': True, 'found': True, 'stocking': True, 'rolled': True, 'ankle': True, 'good': True, 'solution': True, 'go': True, 'standard': True, 'compression': True, '20-30': True, 'stock': True, '114622': True, 'excellent': True, 'support': True, 'stay': True, 'give': True, 'need': True, 'pair': True, 'also': True, 'tore': True, 'struggled': True, 'pull': True, 'time': True, 'riddance/bad': True, 'investment': True}, 'neg')


pos


In [4]:
from random import shuffle

shuffle(pos_set)
shuffle(neg_set)

train_set = pos_set[:208000] + neg_set[:208000]
test_set = pos_set[208000:] + neg_set[208000:]

print(len(train_set)) #416000
print(len(test_set))  #104000

416000
104000


In [5]:
from nltk import classify
from nltk import NaiveBayesClassifier
 
classifier = NaiveBayesClassifier.train(train_set)
 
accuracy = classify.accuracy(classifier, test_set)
print(accuracy * 100)

79.06634615384615


In [6]:
reference = []
for review in test_set:
    reference.append(review[1])
test = []
for review in test_set:
    test.append(classifier.classify(review[0]))

In [7]:
print(len(reference))
print(len(test))

104000
104000


In [8]:
from nltk.metrics import *
print(ConfusionMatrix(reference, test))
CM = ConfusionMatrix(reference, test)

labels = {'pos', 'neg'}

from collections import Counter
TP, FN, FP = Counter(), Counter(), Counter()
for i in labels:
    for j in labels:
        if i == j:
            TP[i] += int(CM[i,j])
        else:
            FN[i] += int(CM[i,j])
            FP[j] += int(CM[i,j])

print("label\tprecision\trecall\tf_measure")
for label in sorted(labels):
    precision, recall = 0, 0
    if TP[label] == 0:
        f_measure = 0
    else:
        precision = float(TP[label]) / (TP[label]+FP[label])
        recall = float(TP[label]) / (TP[label]+FN[label])
        f_measure = float(2) * (precision * recall) / (precision + recall)
    print(label+"\t"+str(precision)+"\t"+str(recall)+"\t"+str(f_measure))
    
print("Accuracy = ",accuracy(reference, test) * 100)

    |     n     p |
    |     e     o |
    |     g     s |
----+-------------+
neg |<47562> 4438 |
pos | 17333<34667>|
----+-------------+
(row = reference; col = test)

label	precision	recall	f_measure
neg	0.7329070036212343	0.9146538461538462	0.813755934813294
pos	0.8865106763840941	0.6666730769230769	0.7610339717907907
Accuracy =  79.06634615384615


In [9]:
review = '''
  Laptop speed is very slow, hanging issue. 
'''
tokenized = word_tokenize(review)
cleaned = clean(tokenized)
print(review, '->', classifier.classify(cleaned))

review = '''
 The product fits as expected. It is comfortable to wear and does not degrade after washing. It is made of good quality cotton material which ensures that it can be worn in summer without feeling warm. Overall a good quality product given the price '''
tokenized = word_tokenize(review)
cleaned = clean(tokenized)
print(review, '->', classifier.classify(cleaned))


  Laptop speed is very slow, hanging issue. 
 -> neg

 The product fits as expected. It is comfortable to wear and does not degrade after washing. It is made of good quality cotton material which ensures that it can be worn in summer without feeling warm. Overall a good quality product given the price  -> pos


In [10]:
print (classifier.show_most_informative_features(10))

Most Informative Features
              restocking = True              neg : pos    =     35.9 : 1.0
         unsubstantiated = True              neg : pos    =     30.3 : 1.0
                  refund = True              neg : pos    =     29.0 : 1.0
               illegible = True              neg : pos    =     28.3 : 1.0
              inoperable = True              neg : pos    =     27.0 : 1.0
                   tripe = True              neg : pos    =     25.4 : 1.0
                unevenly = True              neg : pos    =     24.4 : 1.0
              pleasently = True              pos : neg    =     23.0 : 1.0
           uninteresting = True              neg : pos    =     22.6 : 1.0
               refunding = True              neg : pos    =     21.4 : 1.0
None
